# 使用 FBA 仿真

使用流平衡分析 (flux balance analysis) 的仿真可以使用 `Model.optimize()` 求解. 这会通过目标反应最大化/最小化 (默认最大化) 通量.

In [1]:
import cobra.test
model = cobra.test.create_test_model("textbook")

## 运行 FBA

In [2]:
solution = model.optimize()
print(solution)

<Solution 0.874 at 0x112eb3d30>


Model.optimize() 函数会返回 Solution 对象. 一个 solution 对象有一些属性:

 - `objective_value`: 目标值
 - `status`: 线性规划求解器的状态
 - `fluxes`: 一个通量由反应识别符 indexed 的 pandas series. 反应变量的通量是正方向和逆方向反应变量的原始值的差值.
 - `shadow_prices`: 一个通量由影子价格 indexed 的 pandas series.

比如说, 在最后一次调用 `model.optimize()` 后, 如果优化成功它的状态就是最优的. 如果模型无解，会抛出错误.

In [3]:
solution.objective_value

0.8739215069684307

可用于 cobrapy 的求解器很快，以至于对很多中小型模型，求解会比从求解器中提取值并转换成 Python 对象还要快. 使用 `model.optimize`, 我们从所有反应和代谢物中收集值，重复做这样的事情会花掉很长时间. 如果我们只对单个反应的的通量值或者目标感兴趣, 用 `model.slim_optimize` 会更快，它只进行优化，返回目标值，让你决定是否需要取其他你需要的值。

In [4]:
%%time
model.optimize().objective_value

CPU times: user 3.84 ms, sys: 672 µs, total: 4.51 ms
Wall time: 6.16 ms


0.8739215069684307

In [5]:
%%time
model.slim_optimize()

CPU times: user 229 µs, sys: 19 µs, total: 248 µs
Wall time: 257 µs


0.8739215069684307

### 分析 FBA 的解

使用 FBA 求解的模型可以使用 summary 方法进一步分析, 这会输出一些文字来快速反应模型行为. 对整个模型调用 summary 方法显示模型的输入输出行为, 与其最优目标值.

In [6]:
model.summary()

IN FLUXES        OUT FLUXES    OBJECTIVES
---------------  ------------  ----------------------
o2_e      21.8   h2o_e  29.2   Biomass_Ecol...  0.874
glc__D_e  10     co2_e  22.8
nh4_e      4.77  h_e    17.5
pi_e       3.21


额外地, 单独代谢物的输入输出行为也可以用 summary 方法表示. 比如说, 以下命令可以检查模型整体的氧化还原 (redox balance) 平衡。

In [7]:
model.metabolites.nadh_c.summary()

PRODUCING REACTIONS -- Nicotinamide adenine dinucleotide - reduced (nadh_c)
---------------------------------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  --------------------------------------------------
42%    16     GAPD        g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
24%     9.28  PDH         coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
13%     5.06  AKGDH       akg_c + coa_c + nad_c --> co2_c + nadh_c + succ...
13%     5.06  MDH         mal__L_c + nad_c <=> h_c + nadh_c + oaa_c
8%      3.1   Biomass...  1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0....

CONSUMING REACTIONS -- Nicotinamide adenine dinucleotide - reduced (nadh_c)
---------------------------------------------------------------------------
%       FLUX  RXN ID      REACTION
----  ------  ----------  --------------------------------------------------
100%   38.5   NADH16      4.0 h_c + nadh_c + q8_c --> 3.0 h_e + nad_c + q...


或者来感觉一下主要的能量产生和消耗的反应

In [8]:
model.metabolites.atp_c.summary()

PRODUCING REACTIONS -- ATP (atp_c)
----------------------------------
%      FLUX  RXN ID      REACTION
---  ------  ----------  --------------------------------------------------
67%  45.5    ATPS4r      adp_c + 4.0 h_e + pi_c <=> atp_c + h2o_c + 3.0 h_c
23%  16      PGK         3pg_c + atp_c <=> 13dpg_c + adp_c
7%    5.06   SUCOAS      atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c
3%    1.76   PYK         adp_c + h_c + pep_c --> atp_c + pyr_c

CONSUMING REACTIONS -- ATP (atp_c)
----------------------------------
%      FLUX  RXN ID      REACTION
---  ------  ----------  --------------------------------------------------
76%  52.3    Biomass...  1.496 3pg_c + 3.7478 accoa_c + 59.81 atp_c + 0....
12%   8.39   ATPM        atp_c + h2o_c --> adp_c + h_c + pi_c
11%   7.48   PFK         atp_c + f6p_c --> adp_c + fdp_c + h_c
0%    0.223  GLNS        atp_c + glu__L_c + nh4_c --> adp_c + gln__L_c +...


## 改变目标值

目标函数由目标反应的目标系数属性决定. 一般地, 我们使用一个生物质 ("biomass") 函数，它描述了构建一个细胞的代谢物的构成。

In [9]:
biomass_rxn = model.reactions.get_by_id("Biomass_Ecoli_core")

目前在模型中, 目标中只有一个反应 (生物质反应), 线性系数为 1.

In [10]:
from cobra.util.solver import linear_reaction_coefficients
linear_reaction_coefficients(model)

{<Reaction Biomass_Ecoli_core at 0x112eab4a8>: 1.0}

目标函数可以通过分配 Model.objective 来改变, 这可以是一个反应对象 (或者只是其名字), 或者是一个 `{Reaction: objective_coefficient}` 的 `dict`.

In [11]:
# change the objective to ATPM
model.objective = "ATPM"

# The upper bound should be 1000, so that we get
# the actual optimal value
model.reactions.get_by_id("ATPM").upper_bound = 1000.
linear_reaction_coefficients(model)

{<Reaction ATPM at 0x112eab470>: 1.0}

In [12]:
model.optimize().objective_value

174.99999999999966

我们也可以用更加复杂的目标——包含二次项。

## 运行 FVA

FBA 不会总是给出唯一解, 因为多个通量状态可以达到相同的最优值. FVA (或者 流变异性分析 (flux variability analysis)) 寻找每个代谢物通量最优值的区间.

In [13]:
from cobra.flux_analysis import flux_variability_analysis

In [14]:
flux_variability_analysis(model, model.reactions[:10])

,maximum,minimum
ACALD,-2.208811e-30,-5.247085e-14
ACALDt,0.000000e+00,-5.247085e-14
ACKr,0.000000e+00,-8.024953e-14
ACONTa,2.000000e+01,2.000000e+01
ACONTb,2.000000e+01,2.000000e+01
ACt2r,0.000000e+00,-8.024953e-14
ADK1,3.410605e-13,0.000000e+00
AKGDH,2.000000e+01,2.000000e+01
AKGt2r,0.000000e+00,-2.902643e-14
ALCD2x,0.000000e+00,-4.547474e-14


设置参数 `fraction_of_optimium=0.90` 会给出在 90% 最优性的反应的通量区间.

In [15]:
cobra.flux_analysis.flux_variability_analysis(
    model, model.reactions[:10], fraction_of_optimum=0.9)

,maximum,minimum
ACALD,0.000000e+00,-2.692308
ACALDt,0.000000e+00,-2.692308
ACKr,6.635712e-30,-4.117647
ACONTa,2.000000e+01,8.461538
ACONTb,2.000000e+01,8.461538
ACt2r,0.000000e+00,-4.117647
ADK1,1.750000e+01,0.000000
AKGDH,2.000000e+01,2.500000
AKGt2r,2.651196e-16,-1.489362
ALCD2x,0.000000e+00,-2.333333


标准的 FVA 可能包含循环, 即， 某些高的绝对的通量值，它们只会在参与循环（一种数学上的人造概念，无法实际在体内 (vivo???) 发生）的时候才会高。使用 `loopless` 参数来避免此类循环. 以下, 我们看到 FRD7 and SUCDi 反应会参与循环，但当我们使用无循环 FVA 时，会避免这种事情。

In [16]:
loop_reactions = [model.reactions.FRD7, model.reactions.SUCDi]
flux_variability_analysis(model, reaction_list=loop_reactions, loopless=False)

,maximum,minimum
FRD7,980.0,0.0
SUCDi,1000.0,20.0


In [17]:
flux_variability_analysis(model, reaction_list=loop_reactions, loopless=True)

,maximum,minimum
FRD7,0.0,0.0
SUCDi,20.0,20.0


### 使用 summary 模式运行 FVA

流变异性分析也可以嵌入在 summary 方法中. 比如说, 期望的变异性的基质消耗 (substrate consumption) 和产物的形成 (formation) 可以这样被快速找到

In [18]:
model.optimize()
model.summary(fva=0.95)

IN FLUXES                     OUT FLUXES                    OBJECTIVES
----------------------------  ----------------------------  ------------
id          Flux  Range       id          Flux  Range       ATPM  175
--------  ------  ----------  --------  ------  ----------
o2_e          60  [55.9, 60]  co2_e         60  [54.2, 60]
glc__D_e      10  [9.5, 10]   h2o_e         60  [54.2, 60]
nh4_e          0  [0, 0.673]  for_e          0  [0, 5.83]
pi_e           0  [0, 0.171]  h_e            0  [0, 5.83]
                              ac_e           0  [0, 2.06]
                              acald_e        0  [0, 1.35]
                              pyr_e          0  [0, 1.35]
                              etoh_e         0  [0, 1.17]
                              lac__D_e       0  [0, 1.13]
                              succ_e         0  [0, 0.875]
                              akg_e          0  [0, 0.745]
                              glu__L_e       0  [0, 0.673]


相似地, 代谢物质量平衡的变异性也可以用 FVA 来看.

In [19]:
model.metabolites.pyr_c.summary(fva=0.95)

PRODUCING REACTIONS -- Pyruvate (pyr_c)
---------------------------------------
%       FLUX  RANGE         RXN ID      REACTION
----  ------  ------------  ----------  ----------------------------------------
50%       10  [1.25, 18.8]  PYK         adp_c + h_c + pep_c --> atp_c + pyr_c
50%       10  [9.5, 10]     GLCpts      glc__D_e + pep_c --> g6p_c + pyr_c
0%         0  [0, 8.75]     ME1         mal__L_c + nad_c --> co2_c + nadh_c +...
0%         0  [0, 8.75]     ME2         mal__L_c + nadp_c --> co2_c + nadph_c...

CONSUMING REACTIONS -- Pyruvate (pyr_c)
---------------------------------------
%       FLUX  RANGE         RXN ID      REACTION
----  ------  ------------  ----------  ----------------------------------------
100%      20  [13, 28.8]    PDH         coa_c + nad_c + pyr_c --> accoa_c + c...
0%         0  [0, 8.75]     PPS         atp_c + h2o_c + pyr_c --> amp_c + 2.0...
0%         0  [0, 5.83]     PFL         coa_c + pyr_c --> accoa_c + for_c
0%         0  [0, 1.35]     

在这些 summary 方法中, 值被反应为从中间点 +/- FVA 解的范围, 从最大值与最小值计算而得.

## 运行 pFBA

节俭 (Parsimonious) FBA (经常被写成 pFBA) 寻找这样的通量分布，给出最优的生长速度, 但最小化通量总和. 这牵扯到解两个顺序线性规划问题 (sequential linear programs), 但 cobrapy 可以透明得做处理. 关于 pFBA 的更多细节, 请看 [Lewis et al. (2010)](http://dx.doi.org/10.1038/msb.2010.47).

In [20]:
model.objective = 'Biomass_Ecoli_core'
fba_solution = model.optimize()
pfba_solution = cobra.flux_analysis.pfba(model)

这些函数可以找到大约相同的目标值.

In [21]:
abs(fba_solution.fluxes["Biomass_Ecoli_core"] - pfba_solution.fluxes[
    "Biomass_Ecoli_core"])

7.7715611723760958e-16

## 运行几何 (geometric) FBA

几何 FBA 寻找一个独特的最优通量分布，它在可能的通量区间的中心. 关于几何 FBA 的更多细节, 请看 [K Smallbone, E Simeonidis (2009)](http://dx.doi.org/10.1016/j.jtbi.2009.01.027).

In [22]:
geometric_fba_sol = cobra.flux_analysis.geometric_fba(model)
geometric_fba_sol

,fluxes,reduced_costs
ACALD,1.785214e-14,0.0
ACALDt,1.785214e-14,0.0
ACKr,0.000000e+00,0.0
ACONTa,6.007250e+00,0.0
ACONTb,6.007250e+00,0.0
...,...,...
TALA,1.496984e+00,0.0
THD2,1.522652e-14,0.0
TKT1,1.496984e+00,0.0
TKT2,1.181498e+00,0.0
